In [28]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import keras as k
import nltk
import seaborn as sns
from collections import Counter
%matplotlib inline
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda,GRU,Embedding,LSTM
from keras.optimizers import RMSprop
from keras import backend as K
import keras
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from keras.preprocessing.sequence import pad_sequences

In [29]:
data=pd.read_csv('/home/nancy/Downloads/test_modi.csv',encoding='ISO-8859-1')

In [30]:
data['sequence_length']=0

In [31]:
vocab=pd.read_csv('/home/nancy/Downloads/glove.6B/vocab.csv')

In [32]:
vocab=vocab.drop(vocab.columns[0],axis=1)

In [33]:
vocab=vocab.drop(vocab.columns[1],axis=1)

In [34]:
vocab.index=vocab.index+1

In [35]:
word_id={}
for i in range(len(vocab)):
    word_id[vocab.loc[i+1,'word']]=i+1
    

In [39]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
lemmatizer = WordNetLemmatizer()
#stemming=PorterStemmer()

            
def doc_to_id(h):
    l=[]
    m=[]
    a=nltk.tokenize.word_tokenize(h)
    for w in a:
        try:
            n1=lemmatizer.lemmatize(w.lower())
            #n=stemming.stem(n1)
            l.append(word_id[n1])
        except:
                m.append(w.lower())
    return l,m

In [40]:
input1=[]
for i in range(len(data)):
    a=doc_to_id(data['content'][i])[0]
    input1.append(a)

In [41]:
sequence_length=[]
for i in range(len(data)):
    sequence_length.append(len(input1[i]))

In [42]:
remove_sentence_index=[]
for i in range(len(sequence_length)):
    if(sequence_length[i]>40):
        remove_sentence_index.append(i)


In [43]:
remove2=[]
for i in range(len(sequence_length)):
    if(sequence_length[i]==0):
        remove2.append(i)

In [44]:
data.drop(data.index[remove_sentence_index],inplace=True)

In [45]:
data.drop(data.index[remove2],inplace=True)

In [46]:
data=data.reset_index()

In [47]:
input2=[]
for i in range(len(data)):
    a=doc_to_id(data['content'][i])[0]
    input2.append(a)

In [48]:
sequence_length=[]
for i in range(len(data)):
    sequence_length.append(len(input2[i]))

In [49]:
input_main=pad_sequences(input2,maxlen=40,padding='post')

In [75]:
output=pd.get_dummies(data['sentiment'])

In [51]:
output_main=output.as_matrix()

In [26]:
np.save('/home/nancy/Downloads/input.npy',input_main)
np.save('/home/nancy/Downloads/output.npy',output_main)


In [52]:
word_embedding=pd.read_csv('/home/nancy/Downloads/glove.6B/vocab2.csv')


In [53]:
word_embedding=word_embedding.drop(word_embedding.columns[0:1],axis=1)

In [54]:
word_embedding=word_embedding.as_matrix()

In [55]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(input_main,output_main,test_size=0.10, random_state=42)

In [56]:
model=Sequential()
model.add(Embedding(word_embedding.shape[0],word_embedding.shape[1],input_length=40,mask_zero=True,trainable=False,weights=[word_embedding]))
model.add(LSTM(output_dim=128,go_backwards=True,return_sequences=False,init='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

In [57]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [37]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
class accHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.categorical_accuracy = []

    def on_batch_end(self, batch, logs={}):
        self.categorical_accuracy.append(logs.get('categorical_accuracy'))        

In [59]:
history1=LossHistory()
history2=accHistory()
tensorboard=keras.callbacks.TensorBoard('/home/nancy/Downloads/logs',write_images=True)

In [114]:
model.fit(X_train,y_train,batch_size=128,nb_epoch=15,validation_data=[X_test,y_test],verbose=2,shuffle=True,callbacks=[tensorboard,history1,history2])

Train on 31707 samples, validate on 3523 samples
Epoch 1/15
96s - loss: 1.4773 - categorical_accuracy: 0.4363 - val_loss: 1.5552 - val_categorical_accuracy: 0.4156
Epoch 2/15
110s - loss: 1.4618 - categorical_accuracy: 0.4436 - val_loss: 1.5686 - val_categorical_accuracy: 0.4104
Epoch 3/15
111s - loss: 1.4358 - categorical_accuracy: 0.4530 - val_loss: 1.5700 - val_categorical_accuracy: 0.4056
Epoch 4/15
99s - loss: 1.4167 - categorical_accuracy: 0.4600 - val_loss: 1.5623 - val_categorical_accuracy: 0.4096
Epoch 5/15
119s - loss: 1.3980 - categorical_accuracy: 0.4678 - val_loss: 1.6107 - val_categorical_accuracy: 0.4059
Epoch 6/15
184s - loss: 1.3766 - categorical_accuracy: 0.4762 - val_loss: 1.6374 - val_categorical_accuracy: 0.3965
Epoch 7/15
290s - loss: 1.3499 - categorical_accuracy: 0.4872 - val_loss: 1.6059 - val_categorical_accuracy: 0.3974
Epoch 8/15
272s - loss: 1.3265 - categorical_accuracy: 0.4977 - val_loss: 1.6385 - val_categorical_accuracy: 0.3980
Epoch 9/15
270s - loss: 1

In [95]:
def sentiment(h):
    l=doc_to_id(h)[0]
    m=pad_sequences([l],maxlen=40)
    n=model.predict_classes(m,verbose=0)
    j=output.columns[n]
    print(j[0])

In [91]:
score=model.evaluate(X_test,y_test,verbose=2)

In [90]:
#Model Categorical Accuracy
score[1]

0.74562

happiness
